In [1]:
# why we create prepare_callbacks file??

# this is to set up and prepare Tensorflow keras callbacks(like Tensorboard logging and model checkpointing during training) before the model training starts.

# callbacks are special tool in tensorflow that :
# 1. help monitor training process
# 2. save model at certain points automatically
# 3. adjust learning rate dynamically
# 4. early stopping to prevent overfitting
# 5. log training metrics for visualization
# By setting up these callbacks before training, we ensure that the training process is well-monitored, efficient, and can be analyzed later for performance improvements.

# so, we will create 2 callbacks
# 1. Tensorboard callback : to log training metrics for visualization in Tensorboard
# 2. ModelCheckpoint callback : to save the model at certain points during training

In [2]:
import os

In [3]:
%pwd

'c:\\Users\\guddu\\OneDrive\\Desktop\\DLOOPs Project\\research'

In [4]:
os.chdir("../")

In [5]:
%pwd

'c:\\Users\\guddu\\OneDrive\\Desktop\\DLOOPs Project'

In [6]:

from dataclasses import dataclass 
from pathlib import Path 

@dataclass(frozen=True) 
class PrepareCallbacksConfig:
    root_dir: Path 
    tensorboard_root_log_dir: Path 
    checkpoint_model_filepath: Path

In [7]:

from src.cnnClassifier.constants import * 
from src.cnnClassifier.utils.common import read_yaml , create_directories

In [8]:

# class ---> ConfigurationManager 
# Reads configuration files and preapre PrepareCallbacksConfig for callbacks . 
# It centralizes configuration handling so that all settings come from yaml instead of
# hardcoding paths in the code .

In [9]:
class ConfigurationManager:
    def __init__(
        self, 
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        create_directories([self.config.artifacts_root])


    
    def get_prepare_callback_config(self) -> PrepareCallbacksConfig:
        config = self.config.prepare_callbacks
        model_ckpt_dir = os.path.dirname(config.checkpoint_model_filepath)
        create_directories([
            Path(model_ckpt_dir),
            Path(config.tensorboard_root_log_dir)
        ])

        prepare_callback_config = PrepareCallbacksConfig(
            root_dir=Path(config.root_dir),
            tensorboard_root_log_dir=Path(config.tensorboard_root_log_dir),
            checkpoint_model_filepath=Path(config.checkpoint_model_filepath)
        )

        return prepare_callback_config

In [10]:
import os
import urllib.request as request
from zipfile import ZipFile
import tensorflow as tf
import time

In [11]:

class PrepareCallback:
    def __init__(self, config: PrepareCallbacksConfig):
        self.config = config


    
    @property
    def _create_tb_callbacks(self):
        timestamp = time.strftime("%Y-%m-%d-%H-%M-%S")
        tb_running_log_dir = os.path.join(
            self.config.tensorboard_root_log_dir,
            f"tb_logs_at_{timestamp}",
        )
        return tf.keras.callbacks.TensorBoard(log_dir=tb_running_log_dir)
    

    @property
    def _create_ckpt_callbacks(self):
        return tf.keras.callbacks.ModelCheckpoint(
            filepath=self.config.checkpoint_model_filepath,
            save_best_only=True
        )


    def get_tb_ckpt_callbacks(self):
        return [
            self._create_tb_callbacks,
            self._create_ckpt_callbacks
        ]

In [12]:

try:
    config = ConfigurationManager()
    prepare_callbacks_config = config.get_prepare_callback_config()
    prepare_callbacks = PrepareCallback(config=prepare_callbacks_config)
    callback_list = prepare_callbacks.get_tb_ckpt_callbacks()
    
except Exception as e:
    raise e

✅ YAML Content: {'artifacts_root': 'artifacts', 'data_ingestion': {'root_dir': 'artifacts/data_ingestion', 'source_URL': 'https://github.com/entbappy/Branching-tutorial/raw/master/Chicken-fecal-images.zip', 'local_data_file': 'artifacts/data_ingestion/data.zip', 'unzip_dir': 'artifacts/data_ingestion'}, 'prepare_base_model': {'root_dir': 'artifacts/prepare_base_model', 'base_model_path': 'artifacts/prepare_base_model/base_model.h5', 'updated_base_model_path': 'artifacts/prepare_base_model/base_model_updated.h5'}, 'prepare_callbacks': {'root_dir': 'artifacts/prepare_callbacks', 'tensorboard_root_log_dir': 'artifacts/prepare_callbacks/tensorboard_log_dir', 'checkpoint_model_filepath': 'artifacts/prepare_callbacks/checkpoint_dir/model.h5'}, 'training': {'root_dir': 'artifacts/training', 'trained_model_path': 'artifacts/training/model.h5'}}
[2025-12-06 07:49:16,987: INFO: common: YAML file config\config.yaml loaded successfully]
✅ YAML Content: {'AUGMENTATION': True, 'IMAGE_SIZE': [224, 22

[2025-12-06 07:49:17,000: INFO: common: created directory at: artifacts]
[2025-12-06 07:49:17,005: INFO: common: created directory at: artifacts\prepare_callbacks\checkpoint_dir]
[2025-12-06 07:49:17,008: INFO: common: created directory at: artifacts\prepare_callbacks\tensorboard_log_dir]
